<a href="https://colab.research.google.com/github/ReidelVichot/PUS2022_Heat-TreesPHL/blob/main/MidTermCleaning_11-13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 9.2 MB/s 
     |████████████████████████████████| 6.3 MB 36.3 MB/s 
     |████████████████████████████████| 16.7 MB 3.7 MB/s 


In [2]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Importing the dataframes from the websites
heat_original = gpd.GeoDataFrame.from_file("https://opendata.arcgis.com/api/v3/datasets/ee1f9ce6aa6f41f08fcdfa5101f203d7_0/downloads/data?format=shp&spatialRefId=4326&where=1%3D1")
trees_original = gpd.GeoDataFrame.from_file("https://opendata.arcgis.com/api/v3/datasets/30ef36e9e880468fa74e2d5b18da4cfb_0/downloads/data?format=shp&spatialRefId=4326")



In [4]:
#Checking the shapes of the geodataframes before cleaning them.
print(trees_original.shape, heat_original.shape)

(152453, 7) (384, 11)


In [5]:
#Eliminating creating another dataframe with the columns of interest and without NAN values.
heat = heat_original[["GEOID10", "HVI_SCORE", "geometry"]].dropna().reset_index()
heat.drop(columns=["index"], inplace=True)
trees = trees_original[["geometry"]].dropna().reset_index()
trees.drop(columns=["index"], inplace=True)

#Printing the shapes of the clean dataframes
print(trees.shape, heat.shape)

(152453, 1) (374, 3)


the heat data had 10 rows with NAN that were eliminated

In [6]:
'''
trees.drop(index=5, axis= 0, inplace= True)
trees.reset_index(inplace=True)
trees.drop(columns=["index"], inplace=True)
'''

'\ntrees.drop(index=5, axis= 0, inplace= True)\ntrees.reset_index(inplace=True)\ntrees.drop(columns=["index"], inplace=True)\n'

In [7]:
#len(trees) 152264

In [8]:
'''
heat
heat["trees"] = ""

for i in range(len(heat)):
  ct_trees = 0
  k=0
  s = len(trees)
  while k < s:
    if heat["geometry"].loc[i].contains(trees["geometry"].loc[k]) == True:
      ct_trees = ct_trees + 1
      trees.drop(index=k, axis= 0, inplace= True)
      trees.reset_index(inplace=True)
      trees.drop(columns=["index"], inplace=True)
    k = k+1

    heat["trees"].loc[i] = ct_trees
  print(heat["GEOID10"].loc[i], " was counted");
  

'''


'\nheat\nheat["trees"] = ""\n\nfor i in range(len(heat)):\n  ct_trees = 0\n  k=0\n  s = len(trees)\n  while k < s:\n    if heat["geometry"].loc[i].contains(trees["geometry"].loc[k]) == True:\n      ct_trees = ct_trees + 1\n      trees.drop(index=k, axis= 0, inplace= True)\n      trees.reset_index(inplace=True)\n      trees.drop(columns=["index"], inplace=True)\n    k = k+1\n\n    heat["trees"].loc[i] = ct_trees\n  print(heat["GEOID10"].loc[i], " was counted");\n  \n\n'

In [9]:
# For some reason that I do not know, the intersect is returning a series, 
#so I solved it by creating an array and rejecting the values of the array
#that that have a lenght different than 1.  
array = []
for i in trees.index:
  array.append(heat[heat.geometry.intersects(
        trees.loc[i].geometry)]["GEOID10"])

geoid = []
for i in range(len(array)):
  if len(array[i]) == 1:
    geoid.append(array[i].values[0])
  else:
    geoid.append("")

trees["geoid"] = geoid

In [14]:
ct_trees = trees.groupby("geoid").count()
ct_trees.rename(columns={"geometry":"num_trees"}, inplace=True)

In [15]:
heat = heat.merge(ct_trees, left_on="GEOID10", right_on="geoid")

In [21]:
# This file takes time to be created. I saved it on the google drive so it is easy to access.
# Still it is imporant to merge it with the heat dataframe in order to run the analyses.
# we can use this to merge both dataframes: heat = heat.merge(ct_trees, left_on="GEOID10", right_on="geoid")  
ct_trees.to_csv("/content/drive/MyDrive/Colab Notebooks/PUS22/ct_trees.csv")